# https://www.youtube.com/watch?v=P8tOjiYEFqU

# Import the relevant libraries and load the API key

In [3]:
import pandas as pd
from openai import OpenAI
import os
import ast
import numpy as np
import pdb

In [4]:
# Load the environment with the OpenAI API key
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

True

# Let's try a test call

In [5]:
client = OpenAI()

In [6]:
question = "What temperature do i set my house on Vacation so the pipes don't freeze"
question

"What temperature do i set my house on Vacation so the pipes don't freeze"

In [8]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role":"user",
            "content":question
        }
    ]
)
response

ChatCompletion(id='chatcmpl-BY3R2V6KIrWTKwS4QXzM6RtG8DxKe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It is recommended to set your thermostat to a minimum of 55 degrees Fahrenheit to prevent the pipes from freezing while you are on vacation. You may also want to consider shutting off the water supply to your house if you will be away for an extended period of time.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1747456056, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=22, total_tokens=75, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [9]:
response.choices[0].message.content

'It is recommended to set your thermostat to a minimum of 55 degrees Fahrenheit to prevent the pipes from freezing while you are on vacation. You may also want to consider shutting off the water supply to your house if you will be away for an extended period of time.'

# Let's scrape a website first

In [10]:
# Cell 1: Import additional libraries for scraping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

In [11]:
# Cell 2: Function to scrape a single page
def scrape_page(url):
    """Scrape a single page and return its text content.
    
    Args:
        url: URL to scrape
        
    Returns:
        Dictionary with url and text content
    """
    try:
        print(f"Scraping {url}")
        response = requests.get(url, headers={'User-Agent': 'RAG Scraper'}, timeout=10)
        
        if response.status_code != 200:
            print(f"Failed to fetch {url}: Status code {response.status_code}")
            return None
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove scripts, styles, and other non-content elements
        for tag in soup(['script', 'style', 'meta', 'noscript', 'header', 'footer', 'nav']):
            tag.extract()
        
        # Extract text
        text = soup.get_text(separator='\n')
        lines = (line.strip() for line in text.splitlines())
        text = '\n'.join(line for line in lines if line)
        
        return {
            'url': url,
            'text': text
        }
        
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

In [12]:
# Cell 3: Function to scrape multiple pages
def scrape_website(start_url, max_pages=20):
    """Scrape website pages starting from a URL.
    
    Args:
        start_url: Starting URL
        max_pages: Maximum number of pages to scrape
        
    Returns:
        List of dictionaries with url and text content
    """
    base_domain = urlparse(start_url).netloc
    visited_urls = set()
    urls_to_visit = [start_url]
    scraped_pages = []
    
    while urls_to_visit and len(scraped_pages) < max_pages:
        # Get next URL to scrape
        current_url = urls_to_visit.pop(0)
        
        if current_url in visited_urls:
            continue
            
        visited_urls.add(current_url)
        
        # Scrape the page
        page_data = scrape_page(current_url)
        
        if page_data:
            scraped_pages.append(page_data)
            
            # Parse the page to find more links
            try:
                response = requests.get(current_url, headers={'User-Agent': 'RAG Scraper'}, timeout=10)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                for link in soup.find_all('a', href=True):
                    href = link['href']
                    full_url = urljoin(current_url, href)
                    parsed_url = urlparse(full_url)
                    
                    # Only follow links to the same domain
                    if parsed_url.netloc == base_domain and full_url not in visited_urls and full_url not in urls_to_visit:
                        urls_to_visit.append(full_url)
            except Exception as e:
                print(f"Error finding links on {current_url}: {str(e)}")
        
        # Be polite, don't hammer the server
        time.sleep(1)
    
    print(f"Scraped {len(scraped_pages)} pages")
    return scraped_pages

In [13]:
# Cell 4: Scrape website and save to DataFrame
website_url = "https://khaata.in"  # Replace with your website URL
scraped_data = scrape_website(website_url, max_pages=30)

# Convert to DataFrame
df = pd.DataFrame(scraped_data)

# Save to CSV for backup
df.to_csv('webpage-text.csv', index=False)
print(f"Saved {len(df)} pages to CSV")

# Show sample
df.head()

Scraping https://khaata.in
Scraping https://khaata.in/
Scraping https://khaata.in/aboutus
Scraping https://khaata.in/pricing
Scraping https://khaata.in/blog
Scraping https://khaata.in/faq
Scraping https://khaata.in/login
Scraping https://khaata.in/signup
Scraping https://khaata.in/amazon-to-tally-with-khaata
Scraping https://khaata.in/contact
Scraping https://khaata.in/terms
Scraping https://khaata.in/privacy
Scraping https://khaata.in/signup?next=pricing
Scraping https://khaata.in/blog#tab_1
Scraping https://khaata.in/blog#tab_2
Scraping https://khaata.in/blog/reconcile-your-amazon-sales-with-tally-erp9-with-a-button-click
Scraping https://khaata.in/blog/tag/reconciliation
Scraping https://khaata.in/blog/tag/tally-erp9
Scraping https://khaata.in/blog/how-to-set-up-taxes-on-shopify-india-in-5-steps
Scraping https://khaata.in/blog/tag/shopify
Scraping https://khaata.in/blog/tag/taxes
Scraping https://khaata.in/blog/tag/india
Scraping https://khaata.in/blog/shopify-amazon-flipkart-seller

,url,text
0,https://khaata.in,"Make money\nthis year\n,\nSell fast, Scale fas..."
1,https://khaata.in/,"Make money\nthis year\n,\nSell fast, Scale fas..."
2,https://khaata.in/aboutus,About us\nKhaata is a fully DIY method of impo...
3,https://khaata.in/pricing,Pricing\nFEATURES\nNumber of transactions\nCus...
4,https://khaata.in/blog,Blog\nUnderstanding your transactions takes so...


# Now we will add Retrieval Augmented Generation

## First we create a little vectorDB from all the webpages

In [14]:
question = "Do you have parking"
question

'Do you have parking'

In [68]:
df = pd.read_csv('webpage-text.csv')

In [69]:
df.shape

(30, 2)

In [70]:
df.head()

,url,text
0,https://khaata.in,"Make money\nthis year\n,\nSell fast, Scale fas..."
1,https://khaata.in/,"Make money\nthis year\n,\nSell fast, Scale fas..."
2,https://khaata.in/aboutus,About us\nKhaata is a fully DIY method of impo...
3,https://khaata.in/pricing,Pricing\nFEATURES\nNumber of transactions\nCus...
4,https://khaata.in/blog,Blog\nUnderstanding your transactions takes so...


In [71]:
df['text'].iloc[0]

"Make money\nthis year\n,\nSell fast, Scale faster.\nRequest Demo\nLearn more\nMake money\nthis quarter\n,\nSell fast, Scale faster.\nRequest Demo\nLearn more\nMake money\ntoday.\nSell fast, Scale faster.\nRequest Demo\nLearn more\nWhy Khaata\nKhaata is for everyone who sells products on daily basis and needs to keep track of sales\nQuick & Easy\nAt the click of a button, import all your sales and\nforget about manual entry in Tally. Khaata imports\nnearly 70-100 invoices per minute.\nFlexible and DIY Accounting\nNo typing or manual searching for importing your\nsales data. Finetune it according to your\nrequirement, import for a day, for a week or a\nmonth. You can even import a single invoice.\nSafe & Secure\nData transfer is done in a fully encrypted, secure\nand standardized technology, to ensure complete\nbusiness privacy.\nAll in One Solution\nAll the entries would come with Taxes (CGST,\nSGST, IGST), GSTIN Numbers (for B2B clients),\nStock items and quantities sold or refunded,\

In [72]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n"," ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [73]:
get_embedding(df['text'].iloc[0])[:10]

[-0.006002134643495083,
 0.01511026918888092,
 0.030332466587424278,
 0.0006715675117447972,
 0.041245438158512115,
 -0.03299075365066528,
 -0.03265497088432312,
 0.03318662941455841,
 0.007744012866169214,
 0.018328197300434113]

In [24]:
%%time
df['text'].head(5).apply(get_embedding)

CPU times: user 32.1 ms, sys: 6.47 ms, total: 38.6 ms
Wall time: 6.24 s


0    [-0.006002134643495083, 0.01511026918888092, 0...
1    [-0.006002134643495083, 0.01511026918888092, 0...
2    [-0.016955994069576263, 0.005278302822262049, ...
3    [-0.044205717742443085, -0.014394300058484077,...
4    [-0.01580938883125782, 0.01787036843597889, -0...
Name: text, dtype: object

In [74]:
%%time
# CPU times: user 246 ms, sys: 43 ms, total: 289 ms
# Wall time: 22.1 s
df['embeddings'] = df['text'].apply(get_embedding)
df.to_csv('./website-with-embeddings.csv',index=False)
df.to_pickle('./website-with-embeddings.pkl')

CPU times: user 246 ms, sys: 43 ms, total: 289 ms
Wall time: 22.1 s


In [75]:
%%time
df= pd.read_pickle('./website-with-embeddings.pkl')
df.head()

CPU times: user 6.23 ms, sys: 5.03 ms, total: 11.3 ms
Wall time: 11.1 ms


,url,text,embeddings
0,https://khaata.in,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.006002134643495083, 0.01511026918888092, 0..."
1,https://khaata.in/,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.006002134643495083, 0.01511026918888092, 0..."
2,https://khaata.in/aboutus,About us\nKhaata is a fully DIY method of impo...,"[-0.016955994069576263, 0.005278302822262049, ..."
3,https://khaata.in/pricing,Pricing\nFEATURES\nNumber of transactions\nCus...,"[-0.044205717742443085, -0.014394300058484077,..."
4,https://khaata.in/blog,Blog\nUnderstanding your transactions takes so...,"[-0.01580938883125782, 0.01787036843597889, -0..."


# Second, let's ask a random question to see how relevance in RETRIEVAL works

In [30]:
question_embedding = get_embedding(question)
question, question_embedding[0:10], "..."

('Do you have parking',
 [-0.008038237690925598,
  -0.04131556302309036,
  0.03025338426232338,
  -0.002770782448351383,
  -0.010014624334871769,
  -0.037153277546167374,
  0.016774844378232956,
  0.0029523586854338646,
  -0.032823383808135986,
  -0.02894044853746891],
 '...')

In [59]:
# Get the nearest embeddings to the question_embedding
def distance_calculations(page_embedding):
        return np.dot(page_embedding, question_embedding)
df['distance'] = df['embeddings'].apply(distance_calculations)
df.sort_values('distance',ascending=False, inplace=True)
df

,url,text,embeddings,distance
8,https://khaata.in/amazon-to-tally-with-khaata,Map your transactions in Tally ERP9 accurately...,"[-0.042187683284282684, 0.0015644700033590198,...",0.605783
15,https://khaata.in/blog/reconcile-your-amazon-s...,Reconcile your Amazon sales with Tally ERP9 wi...,"[-0.01810447685420513, 0.000499312300235033, 0...",0.592017
3,https://khaata.in/pricing,Pricing\nFEATURES\nNumber of transactions\nCus...,"[-0.044205717742443085, -0.014394300058484077,...",0.558641
16,https://khaata.in/blog/tag/reconciliation,Blog\nUnderstanding your transactions takes so...,"[-0.016134630888700485, 0.012004275806248188, ...",0.549931
17,https://khaata.in/blog/tag/tally-erp9,Blog\nUnderstanding your transactions takes so...,"[-0.016095511615276337, 0.012006008066236973, ...",0.549914
0,https://khaata.in,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.00600174767896533, 0.015109295025467873, 0...",0.522758
1,https://khaata.in/,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.006002134643495083, 0.01511026918888092, 0...",0.522744
2,https://khaata.in/aboutus,About us\nKhaata is a fully DIY method of impo...,"[-0.016955994069576263, 0.005278302822262049, ...",0.514002
18,https://khaata.in/blog/how-to-set-up-taxes-on-...,How to Set up Taxes on Shopify India in 5 Step...,"[-0.03842793405056, 0.020772458985447884, 0.01...",0.495421
5,https://khaata.in/faq,Frequently Asked Questions\nWhat is Khaata?\nK...,"[0.010821647942066193, 0.007751320023089647, 0...",0.454929


# Third, let's try a relevant question for better distance/relevance

In [76]:
relevant_question = "How do I account for my sales in Tally"

In [77]:
relevant_question_embedding = get_embedding(relevant_question)
relevant_question, relevant_question_embedding[0:10], "..."

('How do I account for my sales in Tally',
 [0.017627380788326263,
  -0.0197730865329504,
  0.06643539667129517,
  -0.01072852872312069,
  -0.023548441007733345,
  -0.03883999213576317,
  0.00656952615827322,
  0.019229870289564133,
  0.0008067616145126522,
  0.03883999213576317],
 '...')

In [78]:
# Get the nearest embeddings to the question_embedding
def distance_calculations(page_embedding):
        return np.dot(page_embedding, relevant_question_embedding)
df['distance'] = df['embeddings'].apply(distance_calculations)
df.sort_values('distance',ascending=False, inplace=True)
df

,url,text,embeddings,distance
8,https://khaata.in/amazon-to-tally-with-khaata,Map your transactions in Tally ERP9 accurately...,"[-0.04198615998029709, 0.001777688623405993, 0...",0.605949
15,https://khaata.in/blog/reconcile-your-amazon-s...,Reconcile your Amazon sales with Tally ERP9 wi...,"[-0.01810981146991253, 0.0004531860176939517, ...",0.592029
3,https://khaata.in/pricing,Pricing\nFEATURES\nNumber of transactions\nCus...,"[-0.044205717742443085, -0.014394300058484077,...",0.558641
17,https://khaata.in/blog/tag/tally-erp9,Blog\nUnderstanding your transactions takes so...,"[-0.016095511615276337, 0.012006008066236973, ...",0.549914
16,https://khaata.in/blog/tag/reconciliation,Blog\nUnderstanding your transactions takes so...,"[-0.016095511615276337, 0.012006008066236973, ...",0.549914
1,https://khaata.in/,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.006002134643495083, 0.01511026918888092, 0...",0.522744
0,https://khaata.in,"Make money\nthis year\n,\nSell fast, Scale fas...","[-0.006002134643495083, 0.01511026918888092, 0...",0.522744
2,https://khaata.in/aboutus,About us\nKhaata is a fully DIY method of impo...,"[-0.016955994069576263, 0.005278302822262049, ...",0.514002
18,https://khaata.in/blog/how-to-set-up-taxes-on-...,How to Set up Taxes on Shopify India in 5 Step...,"[-0.03842793405056, 0.020772458985447884, 0.01...",0.495421
5,https://khaata.in/faq,Frequently Asked Questions\nWhat is Khaata?\nK...,"[0.010821647942066193, 0.007751320023089647, 0...",0.454929


In [79]:
context = df['text'].iloc[0] + "\n" + df['text'].iloc[1] + "\n" + df['text'].iloc[2] + "\n" + df['text'].iloc[3] + "\n"
print(context)

Map your transactions in Tally ERP9 accurately and fast.
Request Demo
How can I fix my amazon accounting quickly and cheaply?
Fix your accounting for amazon sales in under
15 minutes:
How can I fix my amazon accounting quickly and cheaply?
Fix your accounting for amazon sales in under
15 minutes:
What types of problems can I face while
accounting for amazon sales?
When passing amazon sale and purchase entries in Tally ERP9, problems occur when orders are not entered correctly as vouchers in tally sales ledger, and then expenses are not set off against them.
Reconcile all errors that
stop you from filing GSTR-1:
Reconcile all errors that stop you from filing GSTR-1:
Reconcile your Amazon sales with Tally ERP9 with a button click
reconciliation
tally-erp9
Author: Mitalee
Updated : 23-Sep-2023
Share
Here's some great news for everyone wanting to understand if they missed out on any of their Amazon sales or refunds!
Two weeks ago, we unveiled a new feature for instant sales reconciliation 

In [80]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant who is helping Khaata.in, a SaaS product respond to seller questions. When you answer the question, use the first person (e.g. We) to refer to Khaata."},
        {"role": "user", "content": relevant_question},
        {"role": "assistant", "content": f"Use this information from Khaata.in website as context to answer the user question: {context}. Please stick to this context when answering the question."}
    ]
)

In [81]:
response.choices[0].message.content

"In Tally, you can account for your sales by creating a sales voucher for each transaction. Here's how you can do it:\n1. Go to Gateway of Tally and select Accounting Vouchers.\n2. Choose the type of voucher you want to create (e.g., Sales Voucher).\n3. Enter the necessary details such as the party's name, sales ledger account, sales amount, and tax details.\n4. Save the voucher once you have entered all the information accurately.\n5. Repeat these steps for each sales transaction to ensure all your sales are accurately accounted for in Tally.\nRemember to reconcile your Amazon sales with Tally ERP9 using the features available for instant sales reconciliation to ensure all your sales and refunds are imported correctly."

# Now making it repeatable

In [82]:
def query(question): 
    question_embedding = get_embedding(question)

    def distance_calculations(page_embedding):
        return np.dot(page_embedding, question_embedding)
        
    distance_series = df['embeddings'].apply(distance_calculations)
    top_four = distance_series.sort_values(ascending=False).index[0:4]

    text_series = df.loc[top_four]['text']
    context = "\n\n".join(text_series)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistante who is helping Khaata.in, a Saas product respond to seller questions. When you answer the question, use the first person (.e.g We) to refer to Khaata."},
            {"role":"user","content":question},
            {"role": "assistant", "content": f"Use this information from Khaata.in's website as context to answer the user question: {context}. Please stick to this context when answering the question."}
    ])
    
    print(response.choices[0].message.content)

In [83]:
query("How do I import my sales in Tally")

We have launched a new feature that allows you to reconcile your Amazon sales with Tally ERP9 easily. With just a click of a button, you can check if all your Amazon sales and refunds have been imported into Tally. Tally ERP9 will notify you if any invoice needs to be imported, and you can access a full log for reconciliation purposes. This feature enables you to reconcile on a per-day or per-month basis, ensuring accurate and fast import of your sales data into Tally. If you encounter any issues or have further questions, feel free to reach out to us for assistance.


In [84]:
query("How do we account for amazon sales")

We can help you account for Amazon sales by integrating your Amazon sales data with Tally ERP9. With our feature for instant sales reconciliation in Tally, you can easily check if all your Amazon sales and refunds are imported correctly into Tally. This will ensure that you have accurate records of your Amazon transactions in your accounting system. If you encounter any issues or need assistance with mapping your Amazon sales in Tally, please feel free to reach out to us for support.


In [85]:
query("Do you have parking")

I apologize for the confusion, but Khaata.in is a Saas product for accounting and GST filing, so we do not have physical parking facilities. Our services are accessible online, and you can easily manage your invoices and accounts from anywhere with an internet connection. If you have any further questions about our product or services, please feel free to ask.
